In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [46]:
class Hopfield(object):
    def __init__(self, random_state = 1):
        self.rgen = np.random.RandomState(random_state)

    def fit(self, X):
        self.X = X
        self.h = X.shape[1]
        self.w = X.shape[1]

        W_tmp = [[0 for x in range(self.w)] for y in range(self.h)]
        W_tmp = np.array(W_tmp)

        self.W_ = [[0 for x in range(self.w)] for y in range(self.h)]
        self.W_ = np.array(self.W_)

        for a in X: 
            for i in range(self.w):
                for j in range(self.w):
                    W_tmp[i][j] = a[i]*a[j]
            self.W_ = self.W_ + W_tmp
        for i in range(self.w):
            self.W_[i][i] = 0
        return self

    def predict_sync(self, Xi, n_iter = 10):
        self.Xi = Xi
        self.n_iter = n_iter
        self.result = []
        
        W_tmp = [[0 for x in range(self.w)] for y in range(self.h)]
        W_tmp = np.array(W_tmp)
        
        for i in range(self.n_iter):
            for i in range(self.w):
                for j in range(self.w):
                    W_tmp[i][j] = self.W_[i][j] * self.Xi[j]
            
            self.sums = [0]*self.w
            
            for i in range(self.w):
                self.sums[i] = sum(W_tmp[i])
                if self.sums[i] >= 0:
                    self.sums[i] = 1
                else:
                    self.sums[i] = -1
            
            self.Xi = self.sums
            self.result.append(self.Xi)
            find_flag = 0
            
            for l in self.X:
                if np.array_equal(l, self.Xi):
                    find_flag = 1
                    break
            if find_flag == 1:
                break
        return None
    
    def predict_async1(self, Xi, order, n_iter=10):
        self.Xi = Xi
        self.n_iter = n_iter
        self.result = []
        self.order = order

        W_tmp = [[0 for x in range(self.w)] for y in range(self.h)]
        W_tmp = np.array(W_tmp)

        for x in order:
            for i in range(self.w):
                for j in range(self.w):
                    W_tmp[i][j] = self.W_[i][j] * self.Xi[j]

            self.sums = [0]*self.w

            for i in range(self.w):
                self.sums[i] = sum(W_tmp[i])
                if self.sums[i] >= 0:
                    self.sums[i] = 1
                else:
                    self.sums[i] = -1
            
            self.sums = np.array(self.sums)
            self.Xi[x-1] = self.sums[x-1]
            tmp = list(self.Xi)
            self.result.append(tmp)
        return None
    
    def predict_async2(self, Xi, n_iter=10):
        self.order = []
        self.Xi = Xi
        self.n_iter = n_iter
        self.result = []

        for i in range(len(self.Xi)):
            self.order.append(i+1)

        self.order = np.array(self.order) 
        np.random.shuffle(self.order)
        self.order = list(self.order)

        W_tmp = [[0 for x in range(self.w)] for y in range(self.h)]
        W_tmp = np.array(W_tmp)

        for x in self.order:
            for i in range(self.w):
                for j in range(self.w):
                    W_tmp[i][j] = self.W_[i][j] * self.Xi[j]

            self.sums = [0]*self.w

            for i in range(self.w):
                self.sums[i] = sum(W_tmp[i])
                if self.sums[i] >= 0:
                    self.sums[i] = 1
                else:
                    self.sums[i] = -1
            
            self.sums = np.array(self.sums)
            self.Xi[x-1] = self.sums[x-1]
            tmp = list(self.Xi)
            self.result.append(tmp)

        return None

    def show_result(self, width, height, nrows):
        self.width = width
        self.height = height
        self.nrows = nrows

        tmp = []
        for l in self.result:
            tmp.append([l[i:i+width] for i in range(0, len(l), width)])
        if len(tmp) >= nrows:
            divider = len(tmp) / nrows
            divider = int(divider)
        else:
            divider = 1

        _, axs = plt.subplots(nrows, divider)
        axs = axs.flatten()

        for img, ax in zip(tmp, axs):
            ax.imshow(img,cmap='bwr')
        plt.show()

        return None

    def show(self, data, width, height, nrows):
        self.data = np.array(data)
        self.result = data
        self.show_result(width,height,nrows)

In [43]:
X = np.array([
              [-1, 1, -1, 1, -1, 1, 1,-1, 1, -1, 1,-1], 
              [1, 1, 1, -1, 1, -1, -1, 1, -1, -1, 1,-1]], np.int32)

In [48]:
hop = Hopfield()
data = hop.fit(X)
example = [0, 1, 1, 1, 0, 1, -1, -1, -1, 0, 1, 0]

In [ ]:
data.predict_sync(example)
data.show_result(3,4,1)

In [ ]:
data.predict_async1(example,[6 ,1, 11, 12, 2, 9, 3, 5, 7, 10, 4, 8, 8, 11, 2, 9, 7, 5, 1, 4, 6, 12, 3, 10])
data.show_result(3,4,3)

In [ ]:
data.predict_async2([0, 1, 1, 1, 0, 1, -1, -1, -1, 0, 1, 0])
data.show_result(3,4,3)